In [1]:
# Install necessary libraries
!pip install transformers torch matplotlib seaborn tqdm scikit-learn pandas nltk rouge sentence-transformers bitsandbytes accelerate peft

# Import libraries
import json
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
from difflib import SequenceMatcher
from peft import PeftModel, PeftConfig
from accelerate import init_empty_weights, infer_auto_device_map
from transformers import BitsAndBytesConfig
from huggingface_hub import HfApi, HfFolder, whoami

# Import Kaggle secrets for Hugging Face authentication
from kaggle_secrets import UserSecretsClient

# Accessing the Hugging Face token from Kaggle secrets
user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HUGGINGFACE_TOKEN")

# Set Hugging Face API token for authenticating model access
HfFolder.save_token(HUGGINGFACE_TOKEN)

# Download NLTK data for tokenization
nltk.download('punkt')

# Load the dataset from Kaggle's input directory
with open('/kaggle/input/statss/stats_problems_dataset.json', 'r') as f:  # Corrected file path
    dataset = json.load(f)

# Convert dataset to DataFrame for stratified sampling (300-400 problems)
df = pd.DataFrame(dataset)
sampled_dataset = df.groupby('difficulty', group_keys=False).apply(
    lambda x: x.sample(min(len(x), 400 // df['difficulty'].nunique()), random_state=42)
).to_dict('records')

# Load the SentenceTransformer model for embedding similarity
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize ROUGE scorer
rouge = Rouge()

# Function to calculate traditional similarity (BLEU, ROUGE) and embedding similarity
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()

# Function to calculate BLEU score with smoothing
def calculate_bleu_score(reference, hypothesis):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference.split()], hypothesis.split(), smoothing_function=smoothie)

def calculate_rouge(reference, hypothesis):
    scores = rouge.get_scores(hypothesis, reference)
    return scores[0]

def evaluate_answer(generated, correct):
    # If correct is a dictionary, extract the steps and conclusion fields
    if isinstance(correct, dict):
        steps = correct.get('steps', [])
        conclusion = correct.get('conclusion', '')
        correct = ' '.join(steps) + " " + conclusion  # Concatenate steps and conclusion

    # Calculate similarity metrics
    similarity = calculate_similarity(generated, correct)
    bleu_score = calculate_bleu_score(correct, generated)
    rouge_score = calculate_rouge(correct, generated)

    # Embedding-based similarity
    generated_embedding = embedding_model.encode([generated])
    correct_embedding = embedding_model.encode([correct])
    embedding_similarity = cosine_similarity(generated_embedding, correct_embedding)[0][0]

    # Evaluation threshold
    is_correct = embedding_similarity > 0.35  # Threshold can be tuned
    return {
        "is_correct": is_correct,
        "similarity": similarity,
        "embedding_similarity": embedding_similarity,
        "bleu_score": bleu_score,
        "rouge_1": rouge_score['rouge-1']['f'],
        "rouge_2": rouge_score['rouge-2']['f'],
        "rouge_l": rouge_score['rouge-l']['f']
    }

def evaluate_model(model, tokenizer, dataset):
    results = []
    for problem in tqdm(dataset):
        try:
            # Check if solution exists
            if 'solution' not in problem or 'difficulty' not in problem:
                print(f"Skipping problem due to missing key(s)")
                continue

            correct_solution = problem['solution']  # Adjust to handle correct solution data
            prompt = f"Solve the following statistics problem step-by-step, including all necessary calculations and detailed explanations:\n\nProblem: {problem['problem']}\n\nSolution:"
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

            start_time = time.time()
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=350, num_return_sequences=1)
            generated_solution = tokenizer.decode(outputs[0], skip_special_tokens=True)
            end_time = time.time()

            evaluation_results = evaluate_answer(generated_solution, correct_solution)

            results.append({
                "model_name": "CrishN/finetuned_llama",
                "topic": problem.get("topic", "Unknown"),
                "difficulty": problem.get("difficulty", "Unknown"),
                "generated_solution": generated_solution,
                "correct_solution": correct_solution.get('conclusion', ''),
                "time_taken": end_time - start_time,
                **evaluation_results
            })

        except Exception as e:
            print(f"Error processing problem: {e}")

    return results

# Step 1: Load the base LLaMA model and the fine-tuned adapter using the Hugging Face token
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# Load the base model and fine-tuned adapter from Hugging Face
base_model = AutoModelForCausalLM.from_pretrained("unsloth/llama-3-8b-bnb-4bit", token=HUGGINGFACE_TOKEN)
model = PeftModel.from_pretrained(base_model, "CrishN/finetuned_llama", token=HUGGINGFACE_TOKEN)
tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-bnb-4bit", token=HUGGINGFACE_TOKEN)
model.eval()

# Step 2: Evaluate model on the sampled dataset
results = evaluate_model(model, tokenizer, sampled_dataset)

# Step 3: Normalize numpy types
def normalize_types(results):
    for result in results:
        for key, value in result.items():
            if isinstance(value, np.generic):
                result[key] = value.item()
    return results

# Step 4: Normalize and save results
normalized_results = normalize_types(results)

with open('/kaggle/working/llama_finetuned_results.json', 'w') as f:
    json.dump(normalized_results, f)

print("Evaluation complete. Results saved to llama_finetuned_results.json")
torch.cuda.empty_cache()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/tmp/ipykernel_132/1727171284.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_dataset = df.groupby('difficulty', group_keys=False).apply(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 1/320 [00:37<3:20:39, 37.74s/it]

...

100%|█████████▉| 319/320 [3:23:03<00:41, 41.67s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 320/320 [3:23:42<00:00, 38.20s/it]

Evaluation complete. Results saved to llama_finetuned_results.json


In [4]:
from IPython.display import FileLink

# Provide a link to download the file
FileLink('/kaggle/working/llama_finetuned_results.json')

/kaggle/working/llama_finetuned_results.json

In [3]:
with open('/kaggle/working/llama_finetuned_results.json', 'w') as f:
    json.dump(normalized_results, f)

In [ ]:
# Inspect the dataset to understand the structure
for problem in dataset:
    print(problem.keys())  # Print all keys for each problem to see if 'difficulty' and 'conclusion' exist
    break  # Print the keys of the first problem and stop